In [1]:
import opendatasets as od
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from psycopg2.extras import execute_values

In [2]:
DB_NAME = "ai_product_catalog"
DB_USER = "ai_product_catalog"
DB_PASSWORD = "ai_product_catalog123"
DB_HOST = "127.0.0.1"
DB_PORT = "5432"

In [3]:
od.download("https://www.kaggle.com/datasets/sujaykapadnis/products-datasets")

Dataset URL: https://www.kaggle.com/datasets/sujaykapadnis/products-datasets


100%|██████████| 99.7k/99.7k [00:00<00:00, 1.94MB/s]

In [4]:
df = pd.read_csv('products-datasets/Message Group - Product.csv')
df.head()

,S.No,BrandName,Product ID,Product Name,Brand Desc,Product Size,Currancy,MRP,SellPrice,Discount,Category
0,1,4711,FR001,Cologne Fragrance,ekw eau de cologne 400 ml,Small,Rs.,3900,3120,20% off,Fragrance-Women
1,2,109f,DRW1,DRW1 - Westernwear-Women,womens v- neck short dress - yellow,"Size:Medium,Small,X-Large,XX-Large",Rs.,1899,569,70% off,Westernwear-Women
2,3,109f,DRW2,DRW2 - Westernwear-Women,womens round neck solid top - black,"Size:Large,Medium,Small,X-Large",Rs.,1499,599,60% off,Westernwear-Women
3,4,109f,DRW3,DRW3 - Westernwear-Women,womens round neck stripe shift dress - red,"Size:Medium,Small",Rs.,1599,639,60% off,Westernwear-Women
4,5,109f,DRW4,DRW4 - Westernwear-Women,womens round neck solid high low top - black,"Size:Large,Medium,Small,X-Large",Rs.,1199,479,60% off,Westernwear-Women


In [5]:
print ("Shape = " + str(df.shape))
print ("Number of unique Product IDs = " + str(len(df['Product ID'].drop_duplicates())))
print ("Maximum Length of Product ID Column = " + str(df['Product ID'].str.len().max()))
print ("Number of unique Brands = " + str(len(df['BrandName'].drop_duplicates())))
print ("Maximum Length of Brand Column = " + str(df['BrandName'].str.len().max()))
print ("Maximum Length of Product Name Column = " + str(df['Product Name'].str.len().max()))
print ("Maximum Length of Brand Description Column = " + str(df['Brand Desc'].str.len().max()))
print ("Maximum Length of Product Size Column = " + str(df['Product Size'].str.len().max()))
print ("Number of unique Categories = " + str(len(df['Category'].drop_duplicates())))
print ("Maximum Length of Category Column = " + str(df['Category'].str.len().max()))


Shape = (4566, 11)
Number of unique Product IDs = 4566
Maximum Length of Product ID Column = 24
Number of unique Brands = 57
Maximum Length of Brand Column = 23
Maximum Length of Product Name Column = 43
Maximum Length of Brand Description Column = 120
Maximum Length of Product Size Column = 87
Number of unique Categories = 7
Maximum Length of Category Column = 24


In [6]:
categoriesDF = pd.DataFrame({ "Category": df['Category'].drop_duplicates() })
print(categoriesDF.shape)
categoriesDF.head()

(7, 1)


,Category
0,Fragrance-Women
1,Westernwear-Women
117,Indianwear-Women
160,Lingerie&Nightwear-Women
166,Jewellery-Women


In [7]:
with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      execute_values(
          cur = c,
          sql = """
                  INSERT INTO categories(category_desc) VALUES %s
                """,
          argslist = categoriesDF.to_dict(orient="records"),
          template = """
                          ( %(Category)s )
                    """
      )


In [8]:
brandsDF = pd.DataFrame({ "Brand": df['BrandName'].drop_duplicates() })
print(brandsDF.shape)
brandsDF.head()

(57, 1)


,Brand
0,4711
1,109f
115,18.21 man made
116,aarke
161,abercrombie & fitch


In [9]:
with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      execute_values(
          cur = c,
          sql = """
                  INSERT INTO brands(brand_desc) VALUES %s
                """,
          argslist = brandsDF.to_dict(orient="records"),
          template = """
                          ( %(Brand)s )
                    """
      )

In [10]:
df['MRPFloat'] = df['MRP'].replace(['NaN', '#REF!'], None).astype('float')
df['MRPFloat']

0       3900.0
1       1899.0
2       1499.0
3       1599.0
4       1199.0
         ...  
4561    1699.0
4562    1299.0
4563    1299.0
4564    1399.0
4565    2499.0
Name: MRPFloat, Length: 4566, dtype: float64

In [11]:
with psycopg2.connect(database=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) as db_connection:
  with db_connection.cursor() as c:
      execute_values(
          cur = c,
          sql = """
                  INSERT INTO products
                  (
                    sku, 
                    brand_id,
                    product_name,
                    product_desc,
                    size,
                    msrp,
                    category_id
                  ) VALUES %s
                """,
          argslist = df.to_dict(orient="records"),
          template = """
                        (
                            %(Product ID)s,
                            (select brand_id from brands where brand_desc=%(BrandName)s),
                            %(Product Name)s,
                            %(Brand Desc)s,
                            %(Product Size)s,
                            cast(%(MRPFloat)s as double precision),
                            (select category_id from categories where category_desc=%(Category)s)
                        )
                    """
      )